<a href="https://colab.research.google.com/github/dosanchez/data-science-learning/blob/main/exercises_chpt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
df = pd.read_csv('/content/drive/MyDrive/datasets/titanic.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [20]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
df[df["Age"].isna()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q


In [1]:
import os
import chardet
import re
from sklearn.base import BaseEstimator, TransformerMixin

class MailFileImporter (BaseEstimator, TransformerMixin):

    def __init__(self, include_headers = False):
      self.include_headers = include_headers

    def fit(self, directory = ".", spam = False):
      return self

    def transform(self, directory = ".", spam = False):
      directory_path = directory
      mail = []
      # Iterate through all files in the directory
      for filename in sorted(os.listdir(directory_path)):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
          # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            with open(file_path, 'rb') as raw_file:
                raw_data = raw_file.read()
                guessed_encoding = chardet.detect(raw_data)['encoding']
                #print (guessed_encoding)
            # Read the file content
            with open(file_path, 'r', encoding=guessed_encoding) as file:
                content = file.read()
            #print(content)
            #regex indentifies headers.  not body
            pattern = re.compile(r"\n[A-Z]\S*:")
            matches = pattern.finditer(content)
            for i in matches:
              print(i)
      return "finished"

In [2]:
file_path = '/content/drive/MyDrive/datasets/spam/Copy of 0181.4750eb0bfa983cb33844255f8f6a2094'
with open(file_path, 'rb') as raw_file:
  raw_data = raw_file.read()
  guessed_encoding = chardet.detect(raw_data)['encoding']
  print (guessed_encoding)
with open(file_path, 'r', encoding=guessed_encoding) as file:
  content = file.read()
pattern = re.compile(r"\n[A-Z]\S*:") #s[^A-Z]\S*:
matches = pattern.finditer(content)
#for i in matches:
  #print(i.span())
print (dir(pattern))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/datasets/spam/Copy of 0181.4750eb0bfa983cb33844255f8f6a2094'

In [25]:
mails = MailFileImporter().transform("/content/drive/MyDrive/datasets/spam")

<re.Match object; span=(0, 12), match='Return-Path:'>
<re.Match object; span=(0, 12), match='Return-Path:'>


In [24]:
a = "hola verdugo"
b = "verdugo"
print (a - b)

TypeError: unsupported operand type(s) for -: 'str' and 'str'